# Simple Analysis of Experimental Data

In [27]:
import os
import pandas, numpy, matplotlib.pyplot

In [28]:
root: str = '../00-data'
files: str = os.listdir(root)
F: list[pandas.DataFrame] = []
for file in files:
    F.append(pandas.read_csv(os.path.join(root, file)))
    filenameseg = os.path.splitext(file)[0].split('_')
    F[-1]['trial number'] = filenameseg[0]
    F[-1]['browser'] = filenameseg[1]
    F[-1]['push enabled'] = filenameseg[2]
    F[-1]['push freq'] = filenameseg[3]
    if len(filenameseg) > 4:
        F[-1]['push type'] = filenameseg[4]
    else:
        F[-1]['push type'] = ''
    print(f'<{file}>')
    print(F[-1]['Energy trapz (J)'].describe())
    print()

<trial10_chrome_on_high_burst.csv>
count    30.000000
mean     51.972236
std       2.068648
min      48.801221
25%      50.829314
50%      51.462484
75%      53.020023
max      57.473222
Name: Energy trapz (J), dtype: float64

<trial1_chrome_off_idle.csv>
count    30.000000
mean     46.231307
std       2.426237
min      35.072987
25%      45.928497
50%      46.741965
75%      47.259847
max      49.779550
Name: Energy trapz (J), dtype: float64

<trial2_chrome_off_low_even.csv>
count    30.000000
mean     48.062190
std       2.258378
min      40.902735
25%      46.846682
50%      47.755688
75%      49.694906
max      52.517839
Name: Energy trapz (J), dtype: float64

<trial3_chrome_of_low_burst.csv>
count    30.000000
mean     46.470891
std       3.662616
min      31.836826
25%      46.146616
50%      47.540395
75%      48.074592
max      49.775571
Name: Energy trapz (J), dtype: float64

<trial4_chrome_off_high_even.csv>
count    30.000000
mean     57.632000
std       3.423639
min      51